<a href="https://colab.research.google.com/github/donbcolab/composable_vlms/blob/main/notebooks/bccd_finetuned_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q einops flash_attn timm peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import requests
import torch

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Florence-2-base-ft"
peft_model_id = "dwb2023/florence2-finetuned-bccd"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
model.load_adapter(peft_model_id)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Florence-2-large-ft"
peft_model_id = "dwb2023/florence2-large-bccd-base-ft"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, trust_remote_code=True).to(device)
model.load_adapter(peft_model_id)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

adapter_config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

In [ ]:
# model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
# processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

In [ ]:
prompt = "<OD>"

url = "https://huggingface.co/spaces/dwb2023/omniscience/resolve/main/examples/BloodImage_00099_jpg.rf.0a65e56401cdd71253e7bc04917c3558.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)

{'<OD>': {'bboxes': [[507.1999816894531, 124.07999420166016, 628.1599731445312, 236.39999389648438], [376.6399841308594, 187.44000244140625, 485.44000244140625, 303.1199951171875], [429.1199951171875, 15.119999885559082, 539.2000122070312, 117.36000061035156], [124.47999572753906, 217.1999969482422, 223.0399932861328, 330.47998046875], [146.239990234375, 157.1999969482422, 256.32000732421875, 243.1199951171875], [543.0399780273438, 364.0799865722656, 639.6799926757812, 463.44000244140625], [373.44000244140625, 402.47998046875, 500.1600036621094, 479.7599792480469], [1.5999999046325684, 228.239990234375, 50.23999786376953, 329.03997802734375], [56.63999938964844, 337.1999816894531, 143.0399932861328, 417.3599853515625], [139.1999969482422, 347.2799987792969, 239.0399932861328, 449.03997802734375], [303.03997802734375, 25.19999885559082, 388.1600036621094, 107.27999877929688], [225.59999084472656, 366.47998046875, 340.1600036621094, 460.0799865722656], [47.03999710083008, 316.07998657226